In [4]:
import tensorflow as tf
import pandas as pd
import re
import pickle

test_df = pd.read_csv('./test.csv')

## LSTM Demo

In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
model = tf.keras.models.load_model('./LSTM.hdf5')

c:\Users\Soey\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
c:\Users\Soey\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:89: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: Could not interpret initializer identifier: {'module': 'keras.initializers', 'class_name': 'Orthogonal', 'config': {'gain': 1.0, 'seed': None}, 'registered_name': None}

In [20]:
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)
    text = text.lower()
    text = text.strip()
    return text

train_premise = [clean_text(text) for text in test_df["premise"].tolist()]
train_hypothesis = [clean_text(text) for text in test_df["hypothesis"].tolist()]

max_words = 10000
maxlen = 15

# with open('./tokenizer.pickle', 'rb') as handle:
#     tokenizer = pickle.load(handle)

tokenizer.fit_on_texts(train_premise + train_hypothesis)

test_sequences_premise = tokenizer.texts_to_sequences(test_df['premise'])
test_sequences_hypothesis = tokenizer.texts_to_sequences(test_df['hypothesis'])

x_test_premise = pad_sequences(test_sequences_premise, maxlen=maxlen)
x_test_hypothesis = pad_sequences(test_sequences_hypothesis, maxlen=maxlen)

predictions = model.predict([x_test_premise, x_test_hypothesis])
predictions = (predictions > 0.5).astype(int)

ModuleNotFoundError: No module named 'keras.src.preprocessing'

In [ ]:
# Save predictions to CSV
predictions_df = pd.DataFrame(predictions, columns=['prediction'])
predictions_df.to_csv('Group_8_A.csv', index=False)

BERT Demo

In [14]:
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
from transformers import BertTokenizer
model_path = './model_E10_B16_L2e.hdf5'

model = tf.keras.models.load_model(
    model_path,
    custom_objects={'KerasLayer': hub.KerasLayer}
)

ModuleNotFoundError: No module named 'tensorflow_hub'

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

MAX_LEN = 128

test_encode = tokenizer(
    test_df['premise'].tolist(),
    test_df['hypothesis'].tolist(),
    padding='max_length',
    add_special_tokens=True,
    max_length=MAX_LEN,
    truncation=True,
    return_attention_mask=True,
    return_tensors="tf"
)

predictions = model.predict(
    {
        'input_ids': test_encode['input_ids'],
        'attention_mask': test_encode['attention_mask'],
        'token_type_ids': test_encode['token_type_ids']
    }
)
predictions = (predictions > 0.5).astype(int)

In [ ]:
predictions_df = pd.DataFrame(predictions, columns=['prediction'])
predictions_df.to_csv('Group_8_A.csv', index=False)